## **Semi-Supervised Variational Autoencoders (VAE)**

In semi-supervised learning, VAE can leverage the large amounts of unlabeled data while still benefiting from the small set of labeled data.


**Imports**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt


**Data Loading**

In [ ]:
# Generating synthetic data (just for illustration)
X = np.random.randn(1000, 20)
y = np.random.randint(0, 2, size=(1000,))

# Simulating unlabeled data (assigning some labels as -1)
y[::5] = -1  # Assigning -1 (unlabeled) to every 5th sample

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


**Minimal Preprocessing**

In [ ]:
# Normalize the data (if needed)
# No significant preprocessing for this synthetic data


**Model Building**

In [ ]:
# Define a simple VAE model
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2_mean = nn.Linear(512, latent_dim)
        self.fc2_logvar = nn.Linear(512, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 512)
        self.fc4 = nn.Linear(512, input_dim)
        
    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc2_mean(h1), self.fc2_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Initialize VAE
vae = VAE(input_dim=20, latent_dim=5)


**Predictions**

In [ ]:
# Define the loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.BCELoss(reduction='sum')(recon_x, x)
    # Regularization term
    # KL Divergence between Q(z|x) and P(z)
    # (not directly using unlabeled data in this implementation, but can modify for semi-supervised)
    # For simplicity, we'll ignore the exact loss implementation for the semi-supervised aspect here
    return BCE

# Train the model (simplified for illustration)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
epochs = 10
for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {train_loss/len(train_loader.dataset)}")


**Performance Metrics**

In [ ]:
# The performance for VAEs is typically evaluated based on the reconstruction error and KL divergence.
# We can calculate these using the loss function defined earlier.


**Visualizations**

In [ ]:
# Visualize latent space for the trained VAE model
vae.eval()
with torch.no_grad():
    sample = torch.Tensor(X_test)
    reconstructed, _, _ = vae(sample)
    
    # Plot the original vs reconstructed data
    plt.figure(figsize=(8, 6))
    plt.plot(X_test[0, :], label='Original')
    plt.plot(reconstructed[0, :].numpy(), label='Reconstructed')
    plt.legend()
    plt.show()
